In [1]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
import os
import json
from os.path import join
from utils import get_repo_dir, datetime_str, logger, jsonify
from config.config import *
from typing import Union, Tuple
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gc

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch.nn import functional as F

Using personal config for user: suvansh


## TODO
* Add yard line feature
* Add GPU support
* Add hyperparameter sweep

## Some initial variables and data loading

In [2]:
max_num_frames = 30  # truncate if snap-to-throw is > this. units: ds
num_features = 10  # number of channels in input (vx, vy, etc)
out_dir = join(get_repo_dir(), 'output')
os.makedirs(out_dir, exist_ok=True)
models_dir = join(out_dir, 'models')
os.makedirs(models_dir, exist_ok=True)

In [3]:
# # load data
# week1_tracking = pd.read_csv(join(data_dir, 'week1_norm.csv'))
# week1_coverage = pd.read_csv(join(data_dir, 'coverages_week1.csv'), dtype={'coverage': 'category'})

In [4]:
# # merge data
# week1_data = pd.merge(week1_tracking, week1_coverage, how='right', on=['gameId', 'playId'])
# week1_data['coverage_code'] = week1_data.coverage.cat.codes
# num_classes = len(week1_data.coverage.cat.categories)

In [5]:
# load data
weeks = range(1, 18)
tracking_data = pd.concat([pd.read_csv(join(data_dir, f'week{week_num}_norm.csv')) for week_num in tqdm(weeks)], axis=0)
coverage_data = pd.read_csv(join(data_dir, 'coverages_2018.csv')).dropna(subset=['coverage'])
coverage_data.coverage.replace({'3 Seam': 'Cover 3 Zone', 'Cover 1 Double': 'Cover 1 Man'}, inplace=True)
coverage_data = coverage_data[~coverage_data.coverage.isin(['Bracket', 'Mis'])]
coverage_data.coverage = coverage_data.coverage.astype('category')

100%|██████████| 17/17 [01:04<00:00,  3.80s/it]


In [6]:
# merge data
full_data = pd.merge(tracking_data, coverage_data, how='inner', left_on=['gameId', 'playId'], right_on=['game_id', 'play_id'])
full_data['coverage_code'] = full_data.coverage.cat.codes
coverage_label_map = dict(enumerate(full_data['coverage'].cat.categories))
num_classes = len(coverage_label_map)
coverage_label_map

{0: 'Cover 0 Man',
 1: 'Cover 1 Man',
 2: 'Cover 2 Man',
 3: 'Cover 2 Zone',
 4: 'Cover 3 Zone',
 5: 'Cover 4 Zone',
 6: 'Cover 6 Zone',
 7: 'Goal Line',
 8: 'Prevent',
 9: 'Red Zone'}

In [7]:
del tracking_data
gc.collect();

TODO data aug. this is very slightly complicated by the fact that we have precomputed the vx, vy, ax, ay. will have to ask udit about how this is computed since it seems to be inconsistent with v_theta and v_mag?

for now we don't have orientation relative to qb, so I don't think it makes sense to highlight the qb in the way the bdb winner does for rusher, since it would just be distance from qb and qb-relative speed/accel data right now, which I doubt is _extra_ useful for predicting coverage (?). so for now, we'll do 11x11 of off x def with features:
* relative: x, y, vx, vy, ax, ay
* absolute: vx, vy, ax, ay

## Dataset generation

First, we generate the dataset as numpy arrays.

In [8]:
def gen_data():
    grouped = full_data.groupby(['gameId', 'playId'])
    # TODO can change 11s to max number of off and def players in data
    data_X = np.empty((len(grouped), max_num_frames, num_features, 11, 11), dtype=np.float32)  # (P, T, F, D, O): play, frame, feature, def, off
    data_dims = np.empty((len(grouped), 3), dtype=np.int32) # (P, 3) contains (t, d, o): num frames, num def, num off on each play
    data_Y = np.empty(len(grouped), dtype=np.int32)  # (P,)

    valid_plays = 0
    for (game_id, play_id), play_df in tqdm(grouped):
        try:
            first_frame = play_df.loc[(play_df.nflId == 0) & (play_df.event == 'ball_snap')].frameId.iloc[0]
            play_end_frame = play_df.loc[(play_df.nflId == 0) & (play_df.event.isin(['pass_forward', 'pass_shovel', 'qb_sack', 'qb_strip_sack', 'tackle']))].frameId.iloc[0]
        except:
            print(f'({game_id}, {play_id}) failed. events were {play_df.event.unique()}')
            continue
        last_frame = min(first_frame + max_num_frames - 1, play_end_frame)
        play_df = play_df.loc[play_df.frameId.between(first_frame, last_frame)]

        num_def, num_off = 0, 0
        for frame_idx, (frame_id, frame_df) in enumerate(play_df.groupby('frameId')):
            def_ids = frame_df[frame_df.team_pos == 'DEF'].index
            num_def = len(def_ids)
            off_ids = frame_df[frame_df.team_pos == 'OFF'].index
            num_off = len(off_ids)
            if num_def > 11 or num_off > 11:
                print(f'({game_id}, {play_id}), num_def {num_def}, num_off {num_off}')
                break
            if num_def < 3 or num_off < 3:
                print(f'({game_id}, {play_id}), num_def {num_def}, num_off {num_off}')
                break

            outer_sub = np.subtract.outer(
                frame_df.loc[off_ids, ['x', 'y', 'v_x', 'v_y', 'a_x', 'a_y']].values,
                frame_df.loc[def_ids, ['x', 'y', 'v_x', 'v_y', 'a_x', 'a_y']].values
            )
            if np.isnan(outer_sub).any():
                print(f'({game_id}, {play_id}), frame {frame_id} has NaNs')
                break
            # einsum explanation: the two i's get rid of subtraction across cols
            # k before j reorders def before off since output dims in alph. order
            data_X[valid_plays, frame_idx, :6, :num_def, :num_off] = np.einsum('kiji->ijk', outer_sub)
            data_X[valid_plays, frame_idx, -4:, :num_def, :num_off] = frame_df.loc[def_ids, ['v_x', 'v_y', 'a_x', 'a_y']].values.T[...,None]
        if num_def > 11 or num_off > 11 or num_def < 3 or num_off < 3:
            continue
        data_Y[valid_plays] = play_df.coverage_code.iloc[0]
        data_dims[valid_plays] = last_frame - first_frame, num_def, num_off
        valid_plays += 1
    data_X = data_X[:valid_plays]
    data_dims = data_dims[:valid_plays]
    data_Y = data_Y[:valid_plays]
    return data_X, data_dims, data_Y
    

In [9]:
data_save_path = join(out_dir, 'full_data.npz')
# NOTE: uncomment to generate data and save
data_X, data_dims, data_Y = gen_data()
np.savez(data_save_path, x=data_X, dims=data_dims, y=data_Y)
# NOTE: uncomment to load from save
# saved_data = np.load(data_save_path)
# data_X, data_dims, data_Y = saved_data['x'], saved_data['dims'], saved_data['y']

  8%|▊         | 1573/18957 [03:56<31:31,  9.19it/s] 

(2018091605, 2715), num_def 32, num_off 24


 12%|█▏        | 2309/18957 [05:09<24:19, 11.41it/s]

(2018092000, 1539), num_def 0, num_off 6


 13%|█▎        | 2457/18957 [05:24<18:13, 15.09it/s]

...

 98%|█████████▊| 18543/18957 [33:08<00:35, 11.76it/s]

(2018123010, 334), num_def 11, num_off 0


 99%|█████████▊| 18682/18957 [33:21<00:33,  8.11it/s]

2018123012 60 failed. events were: ['None' 'ball_snap']


100%|██████████| 18957/18957 [33:52<00:00,  9.33it/s]


Next, we make a [TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset) out of it.

In [10]:
train_X, test_X, train_dims, test_dims, train_Y, test_Y = train_test_split(
    data_X, data_dims, data_Y, test_size=0.2, random_state=12
)
train_dataset = TensorDataset(*map(torch.tensor, [train_X, train_dims, train_Y]))
test_dataset = TensorDataset(*map(torch.tensor, [test_X, test_dims, test_Y]))

## Model construction

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [12]:
class DeepCoverInner(nn.Module):
    def __init__(self,
                 input_channels: int,
                 output_dim: int,
                 conv_h: Union[int, Tuple[int, int]]=[128, 96],
                 linear_h: Union[int, Tuple[int, int]]=[96, 256]):
        """
        :param input_channels: number of input features
        :param output_dim: dimension of output embedding
        :param conv_h: number of conv channels for each conv block.
            int or tuple of 2 ints.
        :param linear_h: number of hidden units for each linear layer.
            int or tuple of 2 ints.    
        """
        super().__init__()
        if type(conv_h) is int:
            conv_h = (conv_h, conv_h)
        if type(linear_h) is int:
            linear_h = (linear_h, linear_h)
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_channels, conv_h[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(conv_h[0], conv_h[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(conv_h[0], conv_h[0], kernel_size=1),
            nn.ReLU()
        )
        self.bn1 = nn.BatchNorm1d(conv_h[0])
        self.conv_block_2 = nn.Sequential(
            nn.Conv1d(conv_h[0], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h[1]),
            nn.Conv1d(conv_h[1], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h[1]),
            nn.Conv1d(conv_h[1], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h[1]),
        )
        self.conv_h = conv_h
        self.linear_block = nn.Sequential(
            nn.Linear(conv_h[1], linear_h[0]),
            nn.ReLU(),
            nn.BatchNorm1d(linear_h[0]),
            nn.Linear(*linear_h),
            nn.ReLU(),
            nn.LayerNorm(linear_h[1]),
            nn.Linear(linear_h[1], output_dim)
        )
        
    def forward(self, x, dims):
        # let (F', F") and (..., F*) be conv_h and linear_h args to __init__
        orig_shape = x.shape  # (B, T, F, D, O)
        x = x.view(-1, *orig_shape[2:])  # (B*T, F, D, O)
        
        x = self.conv_block_1(x)  # (B*T, F', D, O)
        x = x.view(*orig_shape[:2], *x.shape[1:])  # (B, T, F', D, O)
        
        # this block of code handles variable number of offensive players                
        x_max = torch.stack([
            F.max_pool2d(each[...,:dim[2]], kernel_size=(1, dim[2])).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F', D)
        x_avg = torch.stack([
            F.avg_pool2d(each[...,:dim[2]], kernel_size=(1, dim[2])).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F', D)
        x = (x_max * 0.3 + x_avg * 0.7).view(-1, *x_max.shape[2:])  # (B*T, F', D)
        x = self.bn1(x)

        x = self.conv_block_2(x)
        x = x.view(*orig_shape[:2], *x.shape[1:])  # (B, T, F", D)
        # this block of code handles variable number of defensive players
        x_max = torch.stack([
            F.max_pool1d(each[...,:dim[1]], kernel_size=dim[1].item()).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F")
        x_avg = torch.stack([
            F.avg_pool1d(each[...,:dim[1]], kernel_size=dim[1].item()).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F")
        x = (x_max * 0.3 + x_avg * 0.7).view(-1, *x_max.shape[2:])  # (B*T, F")
        
        x = self.linear_block(x)  # (B*T, F*)
        
        # restore shape
        x = x.view(*orig_shape[:2], -1)  # (B, T, F*)
        return x


class DeepCoverOuterLSTM(nn.Module):
    def __init__(self,
                 input_dim: int,
                 hidden_size: int,
                 num_classes: int,
                 num_layers: int=1,
                 bidirectional: bool=True):
        """
        :param input_dim: input embedding dimension (per frame). same as Inner's output_dim
        :param hidden_size: dimension of LSTM hidden state
        :param num_classes: number of coverage classes
        :param num_layers: number of LSTM layers
        :param bidirectional: whether RNN is bidirectional
        """
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_size, batch_first=True,
                           num_layers=num_layers, bidirectional=bidirectional)
        self.num_layers = num_layers
        self.num_directions = 1 + int(bidirectional)
        self.linear = nn.Sequential(
            nn.Linear(self.num_layers * self.num_directions * hidden_size, num_classes),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x, dims):
        # x is (B, T, F*)
        batch_size = x.shape[0]
        x = nn.utils.rnn.pack_padded_sequence(x, dims[:,0], batch_first=True, enforce_sorted=False)
        x = self.rnn(x)[1][0]  # last hidden state
        x = x.view(self.num_layers, self.num_directions, batch_size, -1)
        x = x.permute(2, 0, 1, 3).reshape(batch_size, -1)  # (B, F^)
        x = self.linear(x)
        return x
    

class DeepCover(nn.Module):
    type_class_map = {
        'LSTM': DeepCoverOuterLSTM
    }
    
    def __init__(self,
                 inner_args: dict,
                 outer_args: dict,
                 outer_type: str='rnn'
                ):
        assert inner_args['output_dim'] == outer_args['input_dim']
        assert (DeepCoverOuter := self.type_class_map.get(outer_type))
        
        super().__init__()
        self.outer = DeepCoverOuter(**outer_args)
        self.inner = DeepCoverInner(**inner_args)
    
    def forward(self, x, dims):
        x = self.inner(x, dims)
        x = self.outer(x, dims)
        return x

In [13]:
# model parameters (TODO sweep?)
model_params = dict(
    embedding_dim=32,
    hidden_size=32,
    bidirectional=True,
    outer_type='LSTM'
)
model = DeepCover(
    inner_args={
        'input_channels': num_features,
        'output_dim': model_params['embedding_dim']
    },
    outer_type=model_params['outer_type'],
    outer_args={
        'input_dim': model_params['embedding_dim'],
        'hidden_size': model_params['hidden_size'],
        'num_classes': num_classes,
        'bidirectional': model_params['bidirectional']
    }
).to(device)

## Training

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
exp_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
dynamic_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=np.sqrt(0.1), patience=8, min_lr=9e-5, verbose=True)
scheduler = dynamic_scheduler

# adding optimizer and scheduler to this so that they get logged
training_params = dict(
    num_epochs=80,
    batch_size=64,
    loss_fn=nn.NLLLoss(),
    save_freq=8,
    optimizer=optimizer,
    scheduler=scheduler
)

train_loader = DataLoader(train_dataset, batch_size=training_params['batch_size'], shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=training_params['batch_size'], shuffle=True, num_workers=1)

In [15]:
def run_loop(model, dataloader, loss_fn, optimizer=None, train=True):
    if train:
        assert optimizer is not None, 'Optimizer must be specified in train mode.'
        model.train()
    else:
        model.eval()
    loss = correct = 0
    for xb, dimb, yb in dataloader:
        xb, dimb, yb = xb.to(device), dimb.to(device), yb.to(device)
        y_pred = model(xb, dimb)
        loss = loss_fn(y_pred, yb.long())
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss += loss.item() * xb.shape[0]
        correct += (torch.max(y_pred, 1)[1] == yb).sum().item()
    acc = correct / len(dataloader.dataset)
    loss /= len(dataloader.dataset)
    return acc, loss

In [ ]:
stats = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
exp_dir = join(models_dir, datetime_str())
os.makedirs(exp_dir)
with open(join(exp_dir, 'variant.json'), 'w') as f:
    jsonify({**model_params, **training_params}, f)
log_to_file, f = True, None
num_epochs = training_params['num_epochs']
batch_size = training_params['batch_size']
loss_fn = training_params['loss_fn']
save_freq = training_params['save_freq']

try:
    best_test_acc = 0
    if log_to_file:
        f = open(join(exp_dir, 'train.log'), 'w')
    for epoch in tqdm(range(num_epochs)):
        """ Training """
        train_acc, train_loss = run_loop(model, train_loader, loss_fn, optimizer, train=True)
        
        """ Evaluation"""
        test_acc, test_loss = run_loop(model, test_loader, loss_fn, train=False)
        
        scheduler.step(test_loss)
        """ Logging """
        stats['train_loss'].append(train_loss)
        stats['test_loss'].append(test_loss)
        stats['train_acc'].append(train_acc)
        stats['test_acc'].append(test_acc)
        logger(f'Epoch {epoch+1:>2}/{num_epochs} | TrLoss {train_loss:>8.5f} | TrAcc {100*train_acc:>5.2f} | TeLoss {test_loss:>8.5f} | TeAcc {100*test_acc:>5.2f}',
              file=f, log_to_file=log_to_file)
        if (epoch >= num_epochs // 4 and (best_test_acc := max(test_acc, best_test_acc)) == test_acc) or \
            (epoch+1) % save_freq == 0 or epoch == num_epochs - 1: 
            # new best after halfway, or save period, or last epoch
            filename = datetime_str() + '.pt'
            torch.save(model.state_dict(), join(exp_dir, filename))
            logger(f'Model saved at {join(exp_dir, filename)}', file=f, log_to_file=log_to_file)
    if log_to_file:
        f.close()
except KeyboardInterrupt as e:
    if input('Do you want to save the model? [y/N]: ').lower()[0] == 'y':
        filename = datetime_str() + '.pt'
        torch.save(model.state_dict(), join(exp_dir, filename))
        logger(f'Model saved at {join(exp_dir, filename)}', file=f, log_to_file=log_to_file)
    raise e

  1%|▏         | 1/80 [18:03<23:47:01, 1083.82s/it]

Epoch  1/80 | TrLoss  0.00117 | TrAcc 71.23 | TeLoss  0.00476 | TeAcc 69.63


  2%|▎         | 2/80 [29:49<21:01:22, 970.29s/it] 

Epoch  2/80 | TrLoss  0.00181 | TrAcc 72.64 | TeLoss  0.00126 | TeAcc 71.95


  4%|▍         | 3/80 [41:13<18:54:56, 884.38s/it]

...

Epoch 12/80 | TrLoss  0.00098 | TrAcc 78.04 | TeLoss  0.00216 | TeAcc 76.76


 16%|█▋        | 13/80 [2:28:19<11:41:16, 628.00s/it]

Epoch 13/80 | TrLoss  0.00066 | TrAcc 78.56 | TeLoss  0.00341 | TeAcc 77.32


 18%|█▊        | 14/80 [2:38:28<11:24:27, 622.24s/it]

Epoch 14/80 | TrLoss  0.00115 | TrAcc 78.25 | TeLoss  0.00157 | TeAcc 76.16


In [ ]:
# trained on week 1
best_model_path = join(models_dir, '03032021_040817/03032021_042056.pt')
best_model_path = join(models_dir, '03042021_070643/03042021_073225.pt')
best_model_path = join(models_dir, '03032021_092502/03032021_100205.pt')
# trained on weeks 1-4
best_model_path = join(models_dir, '03042021_135230/03042021_155248.pt')
best_model_path = join(models_dir, '03042021_173200/03042021_225324.pt')
with open(join(os.path.dirname(best_model_path), 'variant.json'), 'r') as f:
    variant = json.load(f)
best_model = DeepCover(
    inner_args={
        'input_channels': num_features,
        'output_dim': variant['embedding_dim']
    },
    outer_type=variant.get('outer_type', 'LSTM'),  # back compat.
    outer_args={
        'input_dim': variant['embedding_dim'],
        'hidden_size': variant['hidden_size'],
        'num_classes': num_classes,
        'bidirectional': variant['bidirectional']
    }
).to(device)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()
y_preds = []
ys = []
correct = 0
dataloader = test_loader
for xb, dimb, yb in dataloader:
    xb, dimb, yb = xb.to(device), dimb.to(device), yb.to(device)
    y_logits = best_model(xb, dimb)
    y_pred = torch.max(y_logits, 1)[1]
    y_preds.append(y_pred)
    ys.append(yb)
    correct += (y_pred == yb).sum()
acc = correct / len(dataloader.dataset)
print(f'Accuracy: {acc*100:.3f}%')
y_preds = torch.cat(y_preds).flatten()
ys = torch.cat(ys).flatten()
cm = confusion_matrix(ys, y_preds)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
coverage_label_map